The Goal is to predict weapon type used based on the year, target type, country, and attack type

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
import sklearn.model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [28]:
# Read files
df_raw = pd.read_excel('files/GTD/globalterrorismdb_0522dist.xlsx')

In [34]:
df = df_raw[['iyear', 'country_txt', 'attacktype1', 'targtype1', 'weaptype1', 'weaptype1_txt', 'nkill', 'nwound', 'success', 'crit1', 'crit2', 'crit3']]
df = df[df['weaptype1_txt'] != 'Unknown']
df = df[df['weaptype1'] != 'Unknown']
df = df[df['attacktype1'] != 'Unknown']
df = df[df['targtype1'] != 'Unknown']


df = df.dropna().reset_index()

display(df)

,index,iyear,country_txt,attacktype1,targtype1,weaptype1,weaptype1_txt,nkill,nwound,success,crit1,crit2,crit3
0,5,1970,United States,2,3,5,Firearms,0.0,0.0,1,1,1,1
1,6,1970,Uruguay,1,3,5,Firearms,0.0,0.0,0,1,1,1
2,7,1970,United States,3,21,6,Explosives,0.0,0.0,1,1,1,1
3,8,1970,United States,7,4,8,Incendiary,0.0,0.0,1,1,1,1
4,9,1970,United States,7,2,8,Incendiary,0.0,0.0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173340,209699,2020,Yemen,3,12,6,Explosives,0.0,0.0,1,1,1,1
173341,209700,2020,Nepal,7,2,8,Incendiary,0.0,0.0,1,1,1,1
173342,209703,2020,Germany,7,4,8,Incendiary,0.0,0.0,1,1,1,0
173343,209704,2020,Afghanistan,2,14,5,Firearms,1.0,0.0,1,1,1,1


In [39]:
# liss attempt, linear regression 
import pandas as pd
import statsmodels.api as sm

print(df.columns)

# convert string to float
# df['attacktype1'] = df['attacktype1'].astype(float)

# Use one-hot encoding for 'weaptype1_txt'
# df = pd.get_dummies(df, columns=['weaptype1_txt'], prefix='weaptype1')


# Create dummy variables for all categorical variables 
country_dummies = pd.get_dummies(df['country_txt'], prefix='country', drop_first=True)
attack_type_dummies = pd.get_dummies(df['attacktype1'], prefix='attack type', drop_first=True)
target_dummies = pd.get_dummies(df['targtype1'],prefix='target type', drop_first=True)
weapon_dummies = pd.get_dummies(df['weaptype1'],prefix='weapon type', drop_first=True)

# Concatenate the dummy variables with the DataFrame
new_df = pd.concat([df, country_dummies, attack_type_dummies, target_dummies], axis=1)

# Specify the columns to be included in X and ensure all are numeric
X = new_df[['iyear', 'attacktype1', 'weaptype1', 'targtype1'] + 
         attack_type_dummies.columns.tolist() + country_dummies.columns.tolist() + target_dummies.columns.tolist()].astype(float) 
y = new_df[['success']]

# Add a constant to the features to account for the intercept in the model
X_with_const = sm.add_constant(X)

# Fit the model to estimate the time for change regression 
model = sm.OLS(y, X_with_const).fit()
print(model.summary())

Index(['index', 'iyear', 'country_txt', 'attacktype1', 'targtype1',
       'weaptype1', 'weaptype1_txt', 'nkill', 'nwound', 'success', 'crit1',
       'crit2', 'crit3'],
      dtype='object')
                            OLS Regression Results                            
Dep. Variable:                success   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.189
Method:                 Least Squares   F-statistic:                     176.3
Date:                Mon, 04 Mar 2024   Prob (F-statistic):               0.00
Time:                        16:42:28   Log-Likelihood:                -33958.
No. Observations:              173345   AIC:                         6.838e+04
Df Residuals:                  173114   BIC:                         7.070e+04
Df Model:                         230                                         
Covariance Type:            nonrobust                                         
                  

In [23]:
df_encoded = pd.get_dummies(df, columns=['country_txt', 'attacktype1', 'targtype1'])

X = df_encoded.drop('weaptype1_txt', axis=1)
y = df_encoded['weaptype1_txt']
X_intercept = sm.add_constant(X)

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_intercept, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=5).fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5581355101098965


/Users/liss/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
'''
enc = OneHotEncoder()
# Transforming categorical values to matrix
y = enc.fit_transform(y.values.reshape(-1,1))

x_cat = x['country_txt']
x_cat = enc.fit_transform(x_cat.values.reshape(-1,1))
x_cat = x_cat.toarray()

x_num = x.drop(columns={'country_txt'})
x_num = x_num.values

X = np.concatenate([x_cat, x_num], axis = 1)
'''